In [1]:
import numpy as np
import pandas as pd
import sys
import matplotlib.pyplot as plt
import plotly 
import plotly.graph_objs as go

plotly.offline.init_notebook_mode(connected=True)

%matplotlib inline

In [2]:
fname = ("http://www.who.int/entity/healthinfo/global_burden_disease/" +
                        "GHE2015_Deaths-2000-country.xls")
def parse(fname):

    names = ['Sex', 'GHEcode', 'GHEcause', 'Disease Category', 
             'Disease Group', 'Disease Name', 'dclass']

    df = pd.read_excel(fname, sheet_name=1, skiprows=7)
    
    n = names.copy() 
    for column in df.columns[7:]:
        n.append(column)

    df.columns = n
    #names.append(country_code)

    return df 

def Isalpha(s):

    ans = False
    if type(s) != str:
        return ans

    if (len(s) == 2) and ('.' in s)  and s[0].isalpha():
        ans = True
    return ans

def clean_category(df):
    """Do some cleaning on the columns named GHEcause
    Basically drop irrelevant rows used as names"""

    indices = df.index
    i = indices[0]
    subs = df.loc[i, 'Disease Group']
    to_drop = []
    for n in range(len(indices)):
        i = indices[n]
        if ( Isalpha(df.loc[i, 'Disease Category']) ) :
            subs = df.loc[i, 'Disease Group']
            to_drop.append(i)
        df.loc[i, 'Disease Category'] = subs
    df.drop(to_drop, axis=0, inplace=True)

    return df

def clean_dname(df):
    """Do some cleaning on the columns named dname
    Basically drop irrelevant rows used as names"""

    indices = df.index
    df['Dname'] = df['Disease Name'].apply(Isalpha)
    allAs = df['Disease Name']=='a.'
    ind = allAs[allAs==True].index

    to_drop = ind - 1
    for n in range(len(indices)):
        i = indices[n]
        if ( Isalpha(df.loc[i, 'Disease Name']) ) :
            df.loc[i, 'Disease Name'] = df.loc[i, 'dclass']

    df.drop(to_drop, axis=0, inplace=True)
    df.drop(['Disease Group', 'dclass', 'Dname'], axis=1, inplace=True)

    return df

def clean_GHEcause(df):
    
    """The WHO health outcomes are grouped into 3 main categories: 
    I  = Communicable Diseases 
    II = Non Communicable Diseases 
    III= Injuries """

    #get the row index for each of the categories
    I   = df[ df['GHEcause']=='I.' ].index[0]
    II  = df[ df['GHEcause']=='II.' ].index[0]
    III = df[ df['GHEcause']=='III.' ].index[0]

    #Fill up empty space in data frame with category name
    df.loc[0:II-1, 'GHEcause'] = 'Communicable'
    df.loc[II:III-1, 'GHEcause'] = 'NCommunicable'
    df.loc[III:, 'GHEcause'] = 'Injuries'

    #drop these rows, since they only contain summary for each category
    df.drop([I,II,III], axis=0, inplace=True)

    return df

    
def clean(df, sex='Persons'):
     
    #first group the data frame by sex
    df_group = df[ df['Sex'] == sex ].copy() 

    df_group = clean_GHEcause(df_group)
    df_group = clean_category(df_group)
    df_group = clean_dname(df_group)
    df_group.drop(df_group.index[0:2], axis=0, inplace=True)
    
    df_group.replace('.', 0, inplace=True)
                
    return df_group

In [3]:
data = parse(fname)
population = data[data['Disease Category']=="Population ('000) (2)"].copy()
population.drop(
    ['GHEcode', 'GHEcause', 'Disease Category', 'Disease Group', 'Disease Name', 'dclass'],
            axis = 1, inplace = True) 

population.head()
persons = clean(data, sex='Persons')
males = clean(data, sex='Males')
females = clean(data, sex='Females')

In [4]:
#look at population by sex for two countries
population[['Sex', 'AFG', 'CMR']]

,Sex,AFG,CMR
1,Persons,19702,15928
208,Males,10146.5,7933.76
415,Females,9555.4,7993.95


In [5]:
#first few rows of Persons (males+females) data for two countries
names = ['Sex', 'Disease Name', 'AFG', 'CMR']
persons[names].head()

,Sex,Disease Name,AFG,CMR
5,Persons,Tuberculosis,13.576515,10.783075
7,Persons,Syphilis,0.625165,1.851237
8,Persons,Chlamydia,0.000593,0.000889
9,Persons,Gonorrhoea,0.002317,0.002842
10,Persons,Trichomoniasis,0.000000,0.000000


In [6]:
#first few rows of male data for two countries
males[names].head()

,Sex,Disease Name,AFG,CMR
212,Males,Tuberculosis,9.596148,7.261208
214,Males,Syphilis,0.305016,0.930026
215,Males,Chlamydia,0.000000,0.000000
216,Males,Gonorrhoea,0.000687,0.000407
217,Males,Trichomoniasis,0.000000,0.000000


In [7]:
#first few rows of female data for two countries
females[names].head()

,Sex,Disease Name,AFG,CMR
419,Females,Tuberculosis,3.980367,3.521867
421,Females,Syphilis,0.320149,0.921211
422,Females,Chlamydia,0.000593,0.000889
423,Females,Gonorrhoea,0.001629,0.002435
424,Females,Trichomoniasis,0.000000,0.000000


In [8]:
#combine the three groups into one dataframe
frames = [persons, males, females]
frames = pd.concat(frames)

#check that the length of the concatenated dataframe is correct
frames.shape[0]==(persons.shape[0] + males.shape[0] + females.shape[0])

True

In [9]:
#multiply by 1000 since entries in the are per 1000
for column in frames.columns[5:]:
    frames[column] = frames[column] * 1000
frames[names].head()

,Sex,Disease Name,AFG,CMR
5,Persons,Tuberculosis,13576.515427,10783.075092
7,Persons,Syphilis,625.164968,1851.236840
8,Persons,Chlamydia,0.593230,0.888746
9,Persons,Gonorrhoea,2.316669,2.841976
10,Persons,Trichomoniasis,0.000000,0.000000


In [10]:
frames.to_csv('GHE2015_Deaths_2000_country.csv', index=False)

In [11]:
deaths = pd.read_csv('GHE2015_Deaths_2000_country.csv')
deaths.head()

,Sex,GHEcode,GHEcause,Disease Category,Disease Name,AFG,ALB,DZA,AGO,ATG,...,TZA,USA,URY,UZB,VUT,VEN,VNM,YEM,ZMB,ZWE
0,Persons,30.0,Communicable,Infectious and parasitic diseases,Tuberculosis,13576.515427,25.355273,2430.560318,9753.034277,0.911670,...,23706.606999,948.419115,79.774552,4450.752999,24.558486,817.221115,23104.540083,3499.284120,3908.712094,2154.161188
1,Persons,50.0,Communicable,Infectious and parasitic diseases,Syphilis,625.164968,19.760005,153.985797,2059.579872,0.017298,...,4957.218400,164.200926,10.727164,534.347462,1.485421,70.380367,535.758705,1295.026455,663.906762,320.220130
2,Persons,60.0,Communicable,Infectious and parasitic diseases,Chlamydia,0.593230,0.011776,0.306821,2.655487,0.000050,...,3.954588,3.578780,0.224162,0.084461,0.007229,0.405230,1.257119,0.646774,1.537429,1.618422
3,Persons,70.0,Communicable,Infectious and parasitic diseases,Gonorrhoea,2.316669,0.146358,1.186171,9.228348,0.000076,...,20.299861,14.038703,0.720607,0.215157,0.023490,1.437511,9.852171,2.390277,7.618792,5.039567
4,Persons,80.0,Communicable,Infectious and parasitic diseases,Trichomoniasis,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [12]:
males = deaths[ deaths['Sex'] == 'Males' ]
malesUSA = males[['Disease Name', 'USA']].set_index('Disease Name')
malesUSA.sort_values(by='USA',ascending=False, inplace=True)
top10males = malesUSA.loc[malesUSA.index[0:9]]

females = deaths[ deaths['Sex'] == 'Females' ]
femalesUSA = females[['Disease Name', 'USA']].set_index('Disease Name')
femalesUSA.sort_values(by='USA',ascending=False, inplace=True)
top10females = femalesUSA.loc[femalesUSA.index[0:9]]

top10males['USA'].values

array([298373.304037 ,  91208.2591809,  70624.0815272,  44549.635035 ,
        37652.3356935,  36623.0600675,  34646.6937207,  31975.8296422,
        29352.4802225])

In [13]:
fig = {
  "data": [
    {
      "values": list(top10males['USA'].values),
      "labels": list(top10males.index),
      "domain": {"x": [0, .48]},
      "name": "Deaths",
      "hoverinfo":"label+percent+name",
      "hole": .4,
      "type": "pie"
    },
    {
      "values": list(top10females['USA'].values),
      "labels": list(top10females.index),
      "text":["Female"],
      "textposition":"inside",
      "domain": {"x": [.50, 1]},
      "name": "Deaths",
      "hoverinfo":"label+percent+name",
      "hole": .4,
      "type": "pie"
    }],
  "layout": {
        "title":"Cause of Deaths in USA, 2015",
        "annotations": [
            {
                "font": {
                    "size": 20
                },
                "showarrow": False,
                "text": "Males",
                "x": 0.20,
                "y": 0.5
            },
            {
                "font": {
                    "size": 20
                },
                "showarrow": False,
                "text": "Females",
                "x": 0.8,
                "y": 0.5
            }
        ]
    }
}
plotly.offline.iplot(fig, filename='cause-of-death-2015.html')

In [14]:
#d = females.loc[females.index[0:9]]
#d = d[['Sex', 'GHEcode', 'GHEcause', 'Disease Category', 'Disease Name', 'AFG', 'ALB']]
#d.loc[d.index[0:9]].to_csv('GHE2015_Deaths_2000_country.csv', index=False)